# Creating our HH!
## Step 1: Data 🗃️!
We are going to split this step into multiple sub-steps 📄:
1. ⛏️**Web Scraping**.
3. 🐼**Data Transformation**L.
4. 🧹**Data cleansing**.
5. 📤**Export the Data**.
---

# 🤔 Pre-coding
Lets define and importa some useful stuff before we start coding

In [1]:
# Important imports
import requests as req # Library for HTTP requests (allows you to send HTTP requests etremely easily): https://pypi.org/project/requests/
from bs4 import BeautifulSoup # Python Library for pulling data out of HTML files (or in this case, web page): https://www.crummy.com/software/BeautifulSoup/bs4/doc/
import pandas as pd # For Data Analysis and Manipulation in Python: https://pandas.pydata.org/
import numpy as np # For matricial and array operations
import unicodedata # For normalizing in function below

In [2]:
# Normalize text function
def normalize_text(word):
    """
    This function takes a string 'word' and normalized
    Example: hElLó wÓrlD = HELLO WORLD
    """
    word = str(word) # Making sure this is a string
    upper_word = word.upper() # Only upper case letters
    striped_word = upper_word.strip() # No spaces at the beginning or end of the word
    # No accents -> https://docs.python.org/3/library/unicodedata.html#unicodedata.normalize
    normalized_word = ''.join([
        letter for letter in unicodedata.normalize('NFD', striped_word)
        if unicodedata.category(letter) != 'Mn'
    ])

    return normalized_word

# ⛏️Web Scraping 16/12/2022
Is the process of extracting data from a web page, in this case we are going to use my [college page](http://www.dci.ugto.mx/estudiantes/index.php/mcursos/horarios-licenciatura) 🤗

In [3]:
# Request to the page function
def request_url(url):
    res = req.get(url)
    if res.status_code not in range(200, 300):
        raise Exception("Something wen wrong", res.tatus_code)
    else:
        print(res.status_code, "- Everythin is fine 🔥")
    return res

In [4]:
# Check our request function
url = 'http://www.dci.ugto.mx/estudiantes/index.php/mcursos/horarios-licenciatura'
res = request_url(url)
content = res.content
print(content[:15])

200 - Everythin is fine 🔥
b'<!DOCTYPE html>'


In [5]:
# We need a beautiful soup object, not html in a string
soup = BeautifulSoup(content, 'html.parser')
# The schedules are in a table, lets bring it (its in the second one)
all_tables = soup.find_all('table')
schedule_table = all_tables[0] # 👈 THIS IS OUR TABLE
#schedule_table

# 🐼**Data Transformation**: We need to transform to pandas objects to make the cleaning

In [6]:
# We need to check how many columns are there in http://www.dci.ugto.mx/estudiantes/index.php/mcursos/horarios-licenciatura
# So we need each data component in the first row
column_row = schedule_table.find_all('tr')[0] # 1st row
# Every data component
td_with_column_names = column_row.find_all('td')
# Lets have a look to this column names
for td in td_with_column_names:
    print(td.text.replace('\n', ''), end = " | ")
print()
# I am going to change every column name to a better-shorter-normalized column name
column_names = ['index_in_page', 'NAME', 'GROUP', 'DAY/TIME/ROOM1', 'DAY/TIME/ROOM2', 'DAY/TIME/ROOM3', 'DAY/TIME/ROOM4', 'PROFESSORS'] # 👈 THIS IS OUR COLUMN NAMES LIST
for name in column_names:
    print(name, end = ' | ')
    
# Just comparing the number of columns so there is no error
assert len(column_names) == len(td_with_column_names), "Don't have the same lenght! CORRECT IT!"

# | UNIDAD DE APRENDIZAJE  | GRUPO | DÍA/HORA/AULA | DÍA/HORA/AULA | DÍA/HORA/AULA | DÍA/HORA/AULA | PROFESOR | 
index_in_page | NAME | GROUP | DAY/TIME/ROOM1 | DAY/TIME/ROOM2 | DAY/TIME/ROOM3 | DAY/TIME/ROOM4 | PROFESSORS | 

In [7]:
# Now we need to get the actual data IN THE ROWS
all_rows = schedule_table.find_all('tr')
schedules = []
for row in all_rows[1:]: # Starts in 1 because the 0th is the row with the column names
    tds = row.find_all('td') # Need the data of each row
    # We are going to create a dictionary for each row
    d_row = {}
    for index, column in enumerate(column_names):
        d_row[column] = tds[index].text # {column: tds[index]}
        
    # And we need to add it to the schedules list
    schedules.append(d_row)
schedules[23] # 👈 OUR SCHEDULES IN A PSEUDO-JSON FORMAT

{'index_in_page': '24',
 'NAME': 'BIOFÍSICA MÉDICA',
 'GROUP': 'A',
 'DAY/TIME/ROOM1': 'MARTES/10-12/C2',
 'DAY/TIME/ROOM2': 'MIÉRCOLES/10-12/C2',
 'DAY/TIME/ROOM3': '\xa0',
 'DAY/TIME/ROOM4': '\xa0',
 'PROFESSORS': 'MODESTO ANTONIO SOSA AQUINO/VÍCTOR HUGO HERNÁNDEZ GONZÁLEZ'}

In [8]:
# And now we can create a pandas data frame
raw_schedules_df = pd.DataFrame(schedules, columns = column_names)
# I need to save this file 
raw_schedules_df.to_csv('../data/raw_schedules.csv', index=False)
raw_schedules_df.head(3)

,index_in_page,NAME,GROUP,DAY/TIME/ROOM1,DAY/TIME/ROOM2,DAY/TIME/ROOM3,DAY/TIME/ROOM4,PROFESSORS
0,1,ADMINISTRACIÓN Y MANEJO DE PERSONAL,A,SÁBADO/9-13/F6,,,,
1,2,ALEMÁN I,A,SÁBADO/8-11/F7,,,,MA. ISABEL DELGADILLO CANO
2,3,ALGEBRA LINEAL,A,LUNES/8-10/AUDITORIO EDIF. G,MIÉRCOLES/8-10/AUDITORIO EDIF. G,,,MIGUEL ÁNGEL VALLEJO HERNÁNDEZ


# 🧹**Data cleansing**: Lets clean the data and correct some errors
I am gonna list every error and thing as a sub-title in the following cells

**Note:** There are some times that the school gives me the schedules in an excel, so there is no need to scrap the data from the web page, so here I am going to bring that file

In [9]:
# Reading csv document (This covers the case where school gives me an excel file before they update their page)
raw_schedules_og_df = pd.read_csv('../data/raw_schedules.csv')
raw_schedules_og_df.head(10)

,index_in_page,NAME,GROUP,DAY/TIME/ROOM1,DAY/TIME/ROOM2,DAY/TIME/ROOM3,DAY/TIME/ROOM4,PROFESSORS
0,1,ADMINISTRACIÓN Y MANEJO DE PERSONAL,A,SÁBADO/9-13/F6,,,,
1,2,ALEMÁN I,A,SÁBADO/8-11/F7,,,,MA. ISABEL DELGADILLO CANO
2,3,ALGEBRA LINEAL,A,LUNES/8-10/AUDITORIO EDIF. G,MIÉRCOLES/8-10/AUDITORIO EDIF. G,,,MIGUEL ÁNGEL VALLEJO HERNÁNDEZ
3,4,ÁLGEBRA LINEAL,B,MARTES/8-10/F1,JUEVES/8-10/F1,,,TEODORO CÓRDOVA FRAGA
4,5,ÁLGEBRA LINEAL,C,MARTES/10-12/F1,JUEVES/10-12/F1,,,OCTAVIO JOSÉ OBREGÓN DÍAZ
5,6,ÁLGEBRA LINEAL,D,MIÉRCOLES/12-14/F1,VIERNES/12-14/F1,,,ERASTO ORTIZ RICARDO
6,7,ÁLGEBRA LINEAL,E,LUNES/15-17/G1,MIÉRCOLES/15-17/G1,,,JOSÉ DE JESÚS BERNAL ALVARADO
7,8,ANÁLISIS DE CIRCUITOS,A,MARTES/8-10/LAB. DE ELECTRÓNICA EDIF. G,JUEVES/8-12/LAB. DE ELECTRÓNICA EDIF. G,,,CARLOS VILLASEÑOR MORA
8,9,ANÁLISIS DE CIRCUITOS,B,LUNES/12-16/LAB. DE ELECTRÓNICA EDIF.G,MIÉRCOLES/8-10/LAB. DE ELECTRÓNICA EDIF. G,,,JOSÉ MARCO BALLEZA ORDAZ
9,10,ANÁLISIS DE LA CULTURA MEXICANA,A,LUNES/16-19/AUDITORIO DEL EDIF. G,,,,FERNANDO AGUAS ÁNGEL


In [10]:
# Change the columns again because of the case they give me an excel file, remembering I need always this column names:
# index_in_page | NAME | GROUP | DAY/TIME/ROOM1 | DAY/TIME/ROOM2 | DAY/TIME/ROOM3 | DAY/TIME/ROOM4 | PROFESSORS |    

column_names = [
    "index_in_page", 
    "NAME", 
    "GROUP", 
    "DAY/TIME/ROOM1",
    "DAY/TIME/ROOM2",
    "DAY/TIME/ROOM3",
    "DAY/TIME/ROOM4",
    "PROFESSORS",
]

# Just comparing the number of columns so there is no error
assert len(column_names) == len(raw_schedules_og_df.columns), f"Don't have the same lenght!"

In [11]:
# Create the replace dictionary for the rename function for pandas
column_replace_dict = {og_col: new_col for og_col, new_col in zip(raw_schedules_og_df.columns, column_names)}
# Replace the column names
raw_schedules_df = raw_schedules_og_df.rename(columns = column_replace_dict)
raw_schedules_df.head(10)

,index_in_page,NAME,GROUP,DAY/TIME/ROOM1,DAY/TIME/ROOM2,DAY/TIME/ROOM3,DAY/TIME/ROOM4,PROFESSORS
0,1,ADMINISTRACIÓN Y MANEJO DE PERSONAL,A,SÁBADO/9-13/F6,,,,
1,2,ALEMÁN I,A,SÁBADO/8-11/F7,,,,MA. ISABEL DELGADILLO CANO
2,3,ALGEBRA LINEAL,A,LUNES/8-10/AUDITORIO EDIF. G,MIÉRCOLES/8-10/AUDITORIO EDIF. G,,,MIGUEL ÁNGEL VALLEJO HERNÁNDEZ
3,4,ÁLGEBRA LINEAL,B,MARTES/8-10/F1,JUEVES/8-10/F1,,,TEODORO CÓRDOVA FRAGA
4,5,ÁLGEBRA LINEAL,C,MARTES/10-12/F1,JUEVES/10-12/F1,,,OCTAVIO JOSÉ OBREGÓN DÍAZ
5,6,ÁLGEBRA LINEAL,D,MIÉRCOLES/12-14/F1,VIERNES/12-14/F1,,,ERASTO ORTIZ RICARDO
6,7,ÁLGEBRA LINEAL,E,LUNES/15-17/G1,MIÉRCOLES/15-17/G1,,,JOSÉ DE JESÚS BERNAL ALVARADO
7,8,ANÁLISIS DE CIRCUITOS,A,MARTES/8-10/LAB. DE ELECTRÓNICA EDIF. G,JUEVES/8-12/LAB. DE ELECTRÓNICA EDIF. G,,,CARLOS VILLASEÑOR MORA
8,9,ANÁLISIS DE CIRCUITOS,B,LUNES/12-16/LAB. DE ELECTRÓNICA EDIF.G,MIÉRCOLES/8-10/LAB. DE ELECTRÓNICA EDIF. G,,,JOSÉ MARCO BALLEZA ORDAZ
9,10,ANÁLISIS DE LA CULTURA MEXICANA,A,LUNES/16-19/AUDITORIO DEL EDIF. G,,,,FERNANDO AGUAS ÁNGEL


## ❌ 1. Accents and upper letters
We need to normalize every field (we could do this before in the data extract part, but I think is better to separate things correctly)

In [12]:
# Function for normalizing the df with help of our function in 🤔 Pre-coding part: "normalize_text"
def normalize_df(column):
    normalized_column = [] # A list for the normalized column
    for cell in column:
        normalized_column.append(normalize_text(cell))

    return normalized_column

In [13]:
normalized_data = raw_schedules_df.apply(normalize_df)
normalized_data.head(10) # 👈 OUR DATA NORMALIZED

,index_in_page,NAME,GROUP,DAY/TIME/ROOM1,DAY/TIME/ROOM2,DAY/TIME/ROOM3,DAY/TIME/ROOM4,PROFESSORS
0,1,ADMINISTRACION Y MANEJO DE PERSONAL,A,SABADO/9-13/F6,,,,
1,2,ALEMAN I,A,SABADO/8-11/F7,,,,MA. ISABEL DELGADILLO CANO
2,3,ALGEBRA LINEAL,A,LUNES/8-10/AUDITORIO EDIF. G,MIERCOLES/8-10/AUDITORIO EDIF. G,,,MIGUEL ANGEL VALLEJO HERNANDEZ
3,4,ALGEBRA LINEAL,B,MARTES/8-10/F1,JUEVES/8-10/F1,,,TEODORO CORDOVA FRAGA
4,5,ALGEBRA LINEAL,C,MARTES/10-12/F1,JUEVES/10-12/F1,,,OCTAVIO JOSE OBREGON DIAZ
5,6,ALGEBRA LINEAL,D,MIERCOLES/12-14/F1,VIERNES/12-14/F1,,,ERASTO ORTIZ RICARDO
6,7,ALGEBRA LINEAL,E,LUNES/15-17/G1,MIERCOLES/15-17/G1,,,JOSE DE JESUS BERNAL ALVARADO
7,8,ANALISIS DE CIRCUITOS,A,MARTES/8-10/LAB. DE ELECTRONICA EDIF. G,JUEVES/8-12/LAB. DE ELECTRONICA EDIF. G,,,CARLOS VILLASENOR MORA
8,9,ANALISIS DE CIRCUITOS,B,LUNES/12-16/LAB. DE ELECTRONICA EDIF.G,MIERCOLES/8-10/LAB. DE ELECTRONICA EDIF. G,,,JOSE MARCO BALLEZA ORDAZ
9,10,ANALISIS DE LA CULTURA MEXICANA,A,LUNES/16-19/AUDITORIO DEL EDIF. G,,,,FERNANDO AGUAS ANGEL


## ❌ 2. Professors Together
There are some subject with more than 1 professor (because of labs or something else), so we need to generate a field for each professor. When this happens, every profressor is separated with a '/' character, so we can use that for creating the new fields

In [14]:
# Grab the professor list
professor_list = normalized_data['PROFESSORS'].to_list()
# Split all this professors in a matrix
professor_matrix = [professor.split('/') for professor in professor_list]
# We need to re-normalize this matrix because it can be there are some not-wanted spaces
for i in range(len(professor_matrix)):
    for j in range(len(professor_matrix[i])):
        professor_matrix[i][j] = normalize_text(professor_matrix[i][j])
# As an example, lets see some multiple professors
professor_matrix[20:22] # One subject with 1 professor and the next one with 3 professors

[['MODESTO ANTONIO SOSA AQUINO',
  'ARTURO GONZALEZ VEGA',
  'TEODORO CORDOVA FRAGA',
  'FRANCISCO MIGUEL VARGAS LUNA'],
 ['TEODORO CORDOVA FRAGA']]

In [15]:
# We need to know how many create so lets calculate it
max_professors = 0
for professors in professor_matrix:
    max_professors = max(len(professors), max_professors)
    
print("The max of professors per subject is:", max_professors)

The max of professors per subject is: 4


In [16]:
# And we need to add the missing columns as blank to professor matrix so its not sparse
for i in range(len(professor_matrix)):
    for missing in range(max_professors - len(professor_matrix[i])):
        professor_matrix[i].append('')
        
professor_matrix[20:22]

[['MODESTO ANTONIO SOSA AQUINO',
  'ARTURO GONZALEZ VEGA',
  'TEODORO CORDOVA FRAGA',
  'FRANCISCO MIGUEL VARGAS LUNA'],
 ['TEODORO CORDOVA FRAGA', '', '', '']]

In [17]:
# So new fields are:
professor_columns = [f"PROFESSOR{index + 1}" for index in range(max_professors)]
professor_columns

['PROFESSOR1', 'PROFESSOR2', 'PROFESSOR3', 'PROFESSOR4']

In [18]:
# Lets create this fields in our df
normalized_data[professor_columns] = professor_matrix

In [19]:
normalized_data[20:22] # Lets check some records in the df # 👈 THIS IS OUR CORRECT DF FOR CLEANSING

,index_in_page,NAME,GROUP,DAY/TIME/ROOM1,DAY/TIME/ROOM2,DAY/TIME/ROOM3,DAY/TIME/ROOM4,PROFESSORS,PROFESSOR1,PROFESSOR2,PROFESSOR3,PROFESSOR4
20,21,BASES FISICAS PARA EL DIAGNOSTICO POR IMAGENES,A,MARTES/15-18/C2,VIERNES/12-15/C3,,,MODESTO ANTONIO SOSA AQUINO/ARTURO GONZALEZ VE...,MODESTO ANTONIO SOSA AQUINO,ARTURO GONZALEZ VEGA,TEODORO CORDOVA FRAGA,FRANCISCO MIGUEL VARGAS LUNA
21,22,BIOESTADISTICA,A,LUNES/8-10/F8,VIERNES/8-10/F2,,,TEODORO CORDOVA FRAGA,TEODORO CORDOVA FRAGA,,,


## ❌ 3. Empty cells
If there are empty fields (other than DAY/TIME/ROOM fields because empty cells in this fields is normal) then we need to fix it because that means something is wrong with the algorithm

In [20]:
# First of all, lets standard the spaces and blank spaces to np.nan objects
normalized_data = normalized_data.replace(r'^\s*$', np.nan, regex=True)
normalized_data = normalized_data.replace('NAN', np.nan)

In [21]:
normalized_data.head(3) # 👈 STILL OUR DF

,index_in_page,NAME,GROUP,DAY/TIME/ROOM1,DAY/TIME/ROOM2,DAY/TIME/ROOM3,DAY/TIME/ROOM4,PROFESSORS,PROFESSOR1,PROFESSOR2,PROFESSOR3,PROFESSOR4
0,1,ADMINISTRACION Y MANEJO DE PERSONAL,A,SABADO/9-13/F6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,ALEMAN I,A,SABADO/8-11/F7,NaN,NaN,NaN,MA. ISABEL DELGADILLO CANO,MA. ISABEL DELGADILLO CANO,NaN,NaN,NaN
2,3,ALGEBRA LINEAL,A,LUNES/8-10/AUDITORIO EDIF. G,MIERCOLES/8-10/AUDITORIO EDIF. G,NaN,NaN,MIGUEL ANGEL VALLEJO HERNANDEZ,MIGUEL ANGEL VALLEJO HERNANDEZ,NaN,NaN,NaN


In [87]:
# Function to detect if there is an empty cell where it should not be.
def detect_empty_cells(df, columns):
    # Save indices in a list 👇
    empty_cells_index = []
    # Loop for know if it is empty
    for index in range(len(df)):
        for column in columns:
            # print(type(df.at[index, column]), df.at[index, column])
            if pd.isna(df.at[index, column]) or df.at[index, column] == 'NAN' or df.at[index, column] == '':
                empty_cells_index.append([index, column])
                
    return empty_cells_index

In [23]:
# Probemos esta función
columns_to_check = ['NAME', 'GROUP', 'DAY/TIME/ROOM1', 'PROFESSORS', 'PROFESSOR1'] # We only care if these fields are empty becuase that means something is wrong
empty_rows = detect_empty_cells(normalized_data, columns_to_check)

assert not empty_rows, f"It shouldn't be an empty cell in here 👉 {empty_rows}!" # If there are empty rows we need to compare with the link http://www.dci.ugto.mx/estudiantes/index.php/mcursos/horarios-licenciatura
# If this empty data comes from the data source (the page or excel file) we cannot do anything to correct it, it is school obligation

AssertionError: It shouldn't be an empty cell in here 👉 [[0, 'PROFESSORS'], [0, 'PROFESSOR1'], [11, 'PROFESSORS'], [11, 'PROFESSOR1'], [12, 'PROFESSORS'], [12, 'PROFESSOR1'], [13, 'PROFESSORS'], [13, 'PROFESSOR1'], [32, 'PROFESSORS'], [32, 'PROFESSOR1'], [45, 'PROFESSORS'], [45, 'PROFESSOR1'], [46, 'PROFESSORS'], [46, 'PROFESSOR1'], [50, 'PROFESSORS'], [50, 'PROFESSOR1'], [90, 'PROFESSORS'], [90, 'PROFESSOR1'], [91, 'PROFESSORS'], [91, 'PROFESSOR1'], [92, 'PROFESSORS'], [92, 'PROFESSOR1'], [93, 'PROFESSORS'], [93, 'PROFESSOR1'], [95, 'PROFESSORS'], [95, 'PROFESSOR1'], [97, 'PROFESSORS'], [97, 'PROFESSOR1'], [98, 'PROFESSORS'], [98, 'PROFESSOR1'], [99, 'PROFESSORS'], [99, 'PROFESSOR1'], [106, 'DAY/TIME/ROOM1'], [106, 'PROFESSORS'], [106, 'PROFESSOR1'], [107, 'DAY/TIME/ROOM1'], [107, 'PROFESSORS'], [107, 'PROFESSOR1'], [109, 'PROFESSORS'], [109, 'PROFESSOR1'], [110, 'PROFESSORS'], [110, 'PROFESSOR1'], [112, 'PROFESSORS'], [112, 'PROFESSOR1'], [113, 'PROFESSORS'], [113, 'PROFESSOR1'], [11

In [29]:
# Only when we are sure all this records are empty because of the page (and not because of our fault) we can correct them put something like "PENDIENTE"
row = 0
while row < len(empty_rows):
    print(empty_rows[row][0] + 1, empty_rows[row][1])
    if empty_rows[row][1] == 'PROFESSORS':
        row += 1
    row += 1

1 PROFESSORS
12 PROFESSORS
13 PROFESSORS
14 PROFESSORS
33 PROFESSORS
46 PROFESSORS
47 PROFESSORS
51 PROFESSORS
91 PROFESSORS
92 PROFESSORS
93 PROFESSORS
94 PROFESSORS
96 PROFESSORS
98 PROFESSORS
99 PROFESSORS
100 PROFESSORS
107 DAY/TIME/ROOM1
107 PROFESSORS
108 DAY/TIME/ROOM1
108 PROFESSORS
110 PROFESSORS
111 PROFESSORS
113 PROFESSORS
114 PROFESSORS
116 PROFESSORS
118 PROFESSORS
124 PROFESSORS
125 PROFESSORS
138 PROFESSORS
143 PROFESSORS
144 PROFESSORS
145 PROFESSORS
146 PROFESSORS
147 PROFESSORS
149 PROFESSORS
157 PROFESSORS
174 PROFESSORS
181 PROFESSORS
182 PROFESSORS


In [30]:
for row in empty_rows:
    normalized_data.at[row[0], row[1]] = 'PENDIENTE'
    
empty_rows2 = detect_empty_cells(normalized_data, columns_to_check)

assert not empty_rows2, f"It shouldn't be an empty cell in here 👉 {empty_rows2}!" # If there are empty rows we need to compare with the link http://www.dci.ugto.mx/estudiantes/index.php/mcursos/horarios-licenciatura
# If this empty data comes from the data source (the page or excel file) we cannot do anything to correct it, it is school obligation

## ❌ 4. Errors in DAY/TIME/ROOM fields
We need to format this field form, I choose:

_day/start_hour-end_hour/room_

por ejemplo:

_LUNES/14-16/F9_

Lunes is Monday in Spanish and F9 is the class room name.
In the following code we are going to fix the schedules that do not comply with the format 👇.

In [60]:
# Function to detect a bad format in dates (date by date)
def detect_wrong_dates(date):
    # We're not checking NAN values
    if date == 'NAN' or pd.isna(date) or date == 'PENDIENTE':
        return False, None
    # Split for each "/" to obtain every DAY/TIME/ROOM
    date_split = date.split('/')

    # With this we can find 2 possible errors
    # 1. The date field does not have 3 "/"
    # 2. That, the start-end time does not have any "-" 

    # Detect the 1st
    if len(date_split) != 3:
        return True, 'Slash'

    # Detect the 2nd
    hours = date_split[1] # Porque queremos checar la hora
    if hours and len(hours.split('-')) != 2:
        return True, 'Hour'
        
        
    return False, None

In [61]:
# The above 👆 function only works for one date, we need to create a function that runs that 👆 function for all the df
def detect_wrong_dates_in_df(df, date_columns):
    # Iterating over rows
    for index, row in df.iterrows():
        for column in date_columns:
            detection = detect_wrong_dates(row[column])
            if detection[0]:
                print(f'{detection[1]} Error')
                print(f"Index={index}, #Pag={row['index_in_page']}, Column={column[-1]}")

In [62]:
# Test the function
date_columns = [f'DAY/TIME/ROOM{index+1}' for index in range(4)]
detect_wrong_dates_in_df(normalized_data, date_columns) # 😱 Too many errors, there is no option but to fix these by hand ✋
print("------------------------------------------------------")

------------------------------------------------------


If we check out the page http://www.dci.ugto.mx/estudiantes/index.php/mcursos/horarios-licenciatura we can see that the errors are human errors and these errors are no predictable at all, so we need to fix these erros by hand 🥶

### 💨 Fix 10

In [37]:
# 👀 what is happening?
normalized_data.at[10, 'DAY/TIME/ROOM1'] # It doesn't have a slash between the 14 and F2

'MARTES15-17/F8'

In [38]:
# Corrijamos
normalized_data.at[10, 'DAY/TIME/ROOM1'] = "MARTES/15-17/F8" # 'MARTES/15-17/F8'
normalized_data.at[10, 'DAY/TIME/ROOM1']

'MARTES/15-17/F8'

### 💨 Fix 19

In [39]:
# 👀 what is happening?
normalized_data.at[19, 'DAY/TIME/ROOM2']

'VIERNES/15-17F7'

In [40]:
normalized_data.at[19, 'DAY/TIME/ROOM2'] = "VIERNES/15-17/F7"
normalized_data.at[19, 'DAY/TIME/ROOM2']

'VIERNES/15-17/F7'

### 💨 Fix 28

In [41]:
# 👀 what is happening?
normalized_data.at[28, 'DAY/TIME/ROOM4']

'VIERNES/8-11LAB. DE BIOLOGIA EDIF. G'

In [42]:
normalized_data.at[28, 'DAY/TIME/ROOM4'] = 'VIERNES/8-11/LAB. DE BIOLOGIA EDIF. G'
normalized_data.at[28, 'DAY/TIME/ROOM4']

'VIERNES/8-11/LAB. DE BIOLOGIA EDIF. G'

### 💨 Fix 44

In [43]:
# 👀 what is happening?
normalized_data.at[44, 'DAY/TIME/ROOM1']

'MARTES/15-17F2'

In [44]:
normalized_data.at[44, 'DAY/TIME/ROOM1'] = 'MARTES/15-17/F2'
normalized_data.at[44, 'DAY/TIME/ROOM1']

'MARTES/15-17/F2'

### 💨 Fix 47

In [45]:
# 👀 what is happening?
normalized_data.at[47, 'DAY/TIME/ROOM2']

'VIERNES/ 17-19//SALA DE JUNTAS EDIF. B'

In [48]:
normalized_data.at[47, 'DAY/TIME/ROOM2'] = 'VIERNES/17-19/SALA DE JUNTAS EDIF. B'
normalized_data.at[47, 'DAY/TIME/ROOM2']

'VIERNES/17-19/SALA DE JUNTAS EDIF. B'

### 💨 Fix 106 and 107

In [52]:
# 👀 what is happening?
normalized_data.at[106, 'DAY/TIME/ROOM1'] # 👈 No hay nada que hacer aqui, es PENDIENTE

'PENDIENTE'

In [53]:
# 👀 what is happening?
normalized_data.at[107, 'DAY/TIME/ROOM1'] # 👈 No hay nada que hacer aqui, es PENDIENTE

'PENDIENTE'

### 💨 Fix 145

In [54]:
# 👀 what is happening?
normalized_data.at[145, 'DAY/TIME/ROOM2']

'MIERCOLES12-14/C2'

In [55]:
normalized_data.at[145, 'DAY/TIME/ROOM2'] = 'MIERCOLES/12-14/C2'
normalized_data.at[145, 'DAY/TIME/ROOM2']

'MIERCOLES/12-14/C2'

### 💨 Fix 174

In [56]:
# 👀 what is happening?
normalized_data.at[174, 'DAY/TIME/ROOM1']

'JUEVES15-18/AUDITORIO DE EDIF. G'

In [57]:
normalized_data.at[174, 'DAY/TIME/ROOM1'] = 'JUEVES/15-18/AUDITORIO DE EDIF. G'
normalized_data.at[174, 'DAY/TIME/ROOM1']

'JUEVES/15-18/AUDITORIO DE EDIF. G'

### 💨 Fix 175

In [58]:
# 👀 what is happening?
normalized_data.at[175, 'DAY/TIME/ROOM1'] # 👈 No hay nada que hacer aqui, es PENDIENTE

'PENDIENTE'

In [63]:
# Lets look for errors again in case we didn't fix them all
detect_wrong_dates_in_df(normalized_data, date_columns) # ✨ No more date errors

## 🤔 5. Add useful fields
For making things easier in the algorithm part, I need to add some fields, that are already in the table, but in a different way. Example: Separate DAY/TIME/ROOM in a field called DAY, another field called TIME and another field called ROOM.

And, even thought this gonna be akward, I need to add the index (not the index_in_page) for tracking propuses in the database, I am going to add the _ID column

In [80]:
date_column = [column_name for column_name in normalized_data.columns if 'DAY/' in column_name]

for index, column in enumerate(date_column):
    normalized_data[f"DAY{index + 1}"] = list(map(lambda day: day.split('/')[0].strip() if not pd.isna(day) and day  != 'PENDIENTE' and day else np.nan, normalized_data[column].tolist()))
    normalized_data[f"TIME{index + 1}"] = list(map(lambda time: time.split('/')[1].strip() if not pd.isna(time) and time  != 'PENDIENTE' and time else np.nan, normalized_data[column].tolist()))
    normalized_data[f"ROOM{index + 1}"] = list(map(lambda room: room.split('/')[2].strip() if not pd.isna(room) and room  != 'PENDIENTE' and room else np.nan, normalized_data[column].tolist()))

try:
    normalized_data.insert(0, '_ID', range(0, len(normalized_data)))
except:
    pass

normalized_data.head(3)

,_ID,index_in_page,NAME,GROUP,DAY/TIME/ROOM1,DAY/TIME/ROOM2,DAY/TIME/ROOM3,DAY/TIME/ROOM4,PROFESSORS,PROFESSOR1,...,ROOM1,DAY2,TIME2,ROOM2,DAY3,TIME3,ROOM3,DAY4,TIME4,ROOM4
0,0,1,ADMINISTRACION Y MANEJO DE PERSONAL,A,SABADO/9-13/F6,NaN,NaN,NaN,PENDIENTE,PENDIENTE,...,F6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,2,ALEMAN I,A,SABADO/8-11/F7,NaN,NaN,NaN,MA. ISABEL DELGADILLO CANO,MA. ISABEL DELGADILLO CANO,...,F7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,3,ALGEBRA LINEAL,A,LUNES/8-10/AUDITORIO EDIF. G,MIERCOLES/8-10/AUDITORIO EDIF. G,NaN,NaN,MIGUEL ANGEL VALLEJO HERNANDEZ,MIGUEL ANGEL VALLEJO HERNANDEZ,...,AUDITORIO EDIF. G,MIERCOLES,8-10,AUDITORIO EDIF. G,NaN,NaN,NaN,NaN,NaN,NaN


In [91]:
# we need to check out if there is any empty cell again
room_cols = ['ROOM1']
empty_rows3 = detect_empty_cells(normalized_data, room_cols)
empty_rows3

[[22, 'ROOM1'], [106, 'ROOM1'], [107, 'ROOM1'], [175, 'ROOM1']]

In [95]:
for row in empty_rows3:
    normalized_data.at[row[0], row[1]] = 'PENDIENTE'
    
empty_rows3 = detect_empty_cells(normalized_data, columns_to_check)

assert not empty_rows2, f"It shouldn't be an empty cell in here 👉 {empty_rows2}!" # If there are empty rows we need to compare with the link http://www.dci.ugto.mx/estudiantes/index.php/mcursos/horarios-licenciatura
# If this empty data comes from the data source (the page or excel file) we cannot do anything to correct it, it is school obligation

## ❌ 6. Remove the NAN and replace it with a blank space
For the frontend we will need to remove all the NaN data and just to update them to a '' blank space.

In [96]:
#normalized_data = normalized_data.replace(np.nan, '', regex=True)
#normalized_data.head(3)

## ❌ 7. Check the correct data type in each column
We need to be sure that, for example, every professor, name, group and Day field starts with a letter, and every TIME starts and ends with a number.

In [97]:
def check_initial_letter(df, column_name):
    for index, name in enumerate(df[column_name]):
        string_name = str(name)
        print(index, string_name, column_name)
        if not (string_name[0].isalpha() and string_name[-1].isalpha() or string_name[-1].isdigit()):
            return f"The record -> {index} in column -> {column_name} does not start or end with a letter -> {name}"
        
    return False
        
def check_initial_number(df, column_name):
    for index, name in enumerate(df[column_name]):
        string_name = str(name)
        if (string_name and string_name.upper() != 'NAN') and not (string_name[0].isdigit() and string_name[-1].isdigit()):
            return f"The record -> {index} in column -> {column_name} does not start or end with a number -> {name}"
        
    return False


In [98]:
letter_columns = [
    'NAME', 
    'GROUP',
    'DAY/TIME/ROOM1',
    'DAY/TIME/ROOM2',
    'DAY/TIME/ROOM3',
    'DAY/TIME/ROOM4',
    'DAY1',
    'DAY2',
    'DAY3',
    'DAY4',
    'PROFESSORS',
    'PROFESSOR1',
    'PROFESSOR2',
    'PROFESSOR3',
    'PROFESSOR4',
    'ROOM1',
    'ROOM2',
    'ROOM3',
    'ROOM4',
]
number_columns = [
    "_ID",
    "TIME1",
    "TIME2",
    "TIME3",
    "TIME4",
    "index_in_page"
]

ignored_columns= [ # We do not need to check all the data because it is probable we are not going to use all the columns
]

missing = [miss for miss in normalized_data.columns if miss not in letter_columns and miss not in number_columns and miss not in ignored_columns]
# Asserting we are checking all the columns
assert len(letter_columns) + len(number_columns) + len(ignored_columns) == len(normalized_data.columns), f"{len(letter_columns) + len(number_columns)} -> {len(normalized_data.columns)}. \nMissing -> {missing}"

# Check letter data
for column in letter_columns:
    error = check_initial_letter(normalized_data, column)
    if error: print(error)
    
# Check number data
for column in number_columns:
    error = check_initial_number(normalized_data, column)
    if error: print(error)

0 ADMINISTRACION Y MANEJO DE PERSONAL NAME
1 ALEMAN I NAME
2 ALGEBRA LINEAL NAME
3 ALGEBRA LINEAL NAME
4 ALGEBRA LINEAL NAME
5 ALGEBRA LINEAL NAME
6 ALGEBRA LINEAL NAME
7 ANALISIS DE CIRCUITOS NAME
8 ANALISIS DE CIRCUITOS NAME
9 ANALISIS DE LA CULTURA MEXICANA NAME
10 ANALISIS TENSORIAL NAME
11 ANALISIS VECTORIAL NAME
12 ANALISIS VECTORIAL NAME
13 ANALISIS VECTORIAL NAME
14 ANATOMIA Y FISIOLOGIA I NAME
15 ANATOMIA Y FISIOLOGIA II NAME
16 ARQUITECTURA DE MICROCONTROLADORES NAME
17 ASPECTOS BASICOS DEL PROCESO DEL CURTIDO NAME
18 BALANCE DE MATERIA Y ENERGIA NAME
19 BALANCE DE MATERIA Y ENERGIA NAME
20 BASES FISICAS PARA EL DIAGNOSTICO POR IMAGENES NAME
21 BIOESTADISTICA NAME
22 BIOFISICA NAME
23 BIOFISICA MEDICA NAME
24 BIOLOGIA CELULAR NAME
25 BIOLOGIA CELULAR NAME
26 BIOMECANICA NAME
27 BIOMECANICA NAME
28 BIOQUIMICA NAME
29 BIOSEGURIDAD NAME
30 BIOTECNOLOGIA NAME
31 CALCULO DE VARIAS VARIABLES NAME
32 CALCULO DE VARIAS VARIABLES NAME
33 CALCULO DE VARIAS VARIABLES NAME
34 CALCULO DIF

IndexError: string index out of range

In [99]:
normalized_data.at[104, 'PROFESSORS']

'ARTURO VEGA GONZALEZ/'

In [52]:
# Correct this errors (some of them are not errors just database inconsistent stuff)
normalized_data.at[104, 'PROFESSORS'] = 'ARTURO VEGA GONZALEZ'
normalized_data.at[104, 'PROFESSORS']

'ARTURO VEGA GONZALEZ'

# 📤**Export the Data**
We are done with the data analysis and cleaning, we can now export this file to create the algorithm for create the schedule combinations

In [53]:
# In JSON because that is how the data is going to be loaded from the request to the API
normalized_data.to_json('../data/clean_schedules.json', orient = 'records')
# Lets import in csv because we are going to use airtable and that is the format used there
normalized_data.to_csv('../data/clean_schedules.csv', index = False)

# 🤔 What is next?
Now that I have the clean data in a CSV format, this file is going to be used to create (or update) a table in Airtable, which I am going to use in the following steps for creating the algorithm for making the schedule combinations.